<a href="https://colab.research.google.com/github/MovinduSenura/Chest-Xray-Pneumonia-Detection/blob/dev-sayuri/notebooks/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Imports

import os

In [4]:
# Clone GITHUB Repository

if not os.path.exists('Chest-Xray-Pneumonia-Detection'):
    !git clone https://github.com/MovinduSenura/Chest-Xray-Pneumonia-Detection.git

Cloning into 'Chest-Xray-Pneumonia-Detection'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 103 (delta 22), reused 5 (delta 5), pack-reused 68 (from 2)
Receiving objects: 100% (103/103), 49.90 KiB | 3.84 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [5]:
#Go into the project directory

%cd Chest-Xray-Pneumonia-Detection


/content/Chest-Xray-Pneumonia-Detection


In [23]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Dataset Location

DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/chest_xray'

In [ ]:
# Create TRAIN/VAL/TEST Generators

from notebooks.preprocessing import create_generators
train_gen, val_gen, test_gen = create_generators(DATA_DIR)
print("✅ Data generators created successfully!\n")